## Import ERA-5 Total Precipitation and downsample from hourly to daily and monthly 

In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
import xarray as xr
import flox
import glob
import os

/Users/katherinedixon/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
def sum_df(dataset):
    dataset.tp.attrs['units'] = 'm'
    sum_daily = dataset.resample(time='D').sum(dim='time')
    
    sum_daily = sum_daily.rename({'tp':'sum_tp'})
    
    # faster to do this on the xarray
    sum_daily['year'] = sum_daily['time'].dt.strftime('%Y')
    sum_daily['month'] = sum_daily['time'].dt.strftime('%B')
    sum_daily['day'] = sum_daily['time'].dt.strftime('%d')
    
    df = sum_daily.to_dataframe()
    df = df.reset_index()
    
    return(df)

In [3]:
model_dir = "data/"

In [4]:
state = "Colorado"
state_code = "CO" # "AZ_new"
yr1 = 2000
yr2 = 2009

In [5]:
f1 = xr.open_dataset(model_dir + "pr/" + state_code+"_pr_" + str(yr1) + "-" + str(yr2) + ".nc", decode_times = True)

In [6]:
df1 = sum_df(f1)

In [7]:
df_all = pd.concat([df1])

In [8]:
df_all.to_csv("downsampled/"+state+"_pr_" + str(yr1) + "-" + str(yr2) + ".csv")

### Summary statistics- Converting from daily to monthly total precipitation

In [9]:
year1 = '2000'
year2 = '2009'

In [10]:
def downsample_sums(filename):

    dataset = xr.open_dataset(filename)

    dataset.tp.attrs['units'] = 'm'
    sum_daily = dataset.resample(time='D').sum(dim='time')
    
    sum_daily = sum_daily.rename({'tp':'sum_tp'})
    
    # faster to do this on the xarray
    sum_daily['year'] = sum_daily['time'].dt.strftime('%Y')
    sum_daily['month'] = sum_daily['time'].dt.strftime('%B')
    sum_daily['day'] = sum_daily['time'].dt.strftime('%d')
    
    df = sum_daily.to_dataframe()
    df = df.reset_index()

    df_means = df.groupby(['latitude','longitude','month','year'])[['sum_tp']].sum()
    df_means = df_means.reset_index()
    
    return(df_means)

In [11]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [13]:
filenames = listdir_nohidden(model_dir + "pr/")

pr_df = pd.DataFrame()

for file in filenames:

    pr_df = pr_df.append(downsample_sums(file))

pr_df.to_csv("downsampled/means_pr_" + year1 + "-" + year2 + "_ERA5.csv")
